Membuat variasi word 2 vector dengan source textrank pke, yg bisa mengolah phrase N-gram
step-step yg akan dilakukan antara lain 
1. Imports & Setup
2. Load Dataset:
3. Preprocess Data:
4. Word2Vec Processing:
5. TextRank Integration:
6. Graph Construction with Word2Vec:
7. Extract Keyphrases:
8. Post-Processing & Results Display:

1. Imports & Setup

In [12]:
#1. rutin1 import module
import pandas as pd
import os
import sys
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

2. Load Dataset

In [14]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "notebooks/postager_nlp-id/dataset_ekstraksi_r29_pos_sm.xlsx")
df = pd.read_excel(dataset_path)
df_pos = df['pos_sentence_list']

3. Preprocess Data:

In [15]:
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary

def preprocess(text):
    '''
    fungsi untuk menghilangkan karakter yg tidak bermakna dan menghilangkan stopword.
    referensi stopword: tala + sastrawi + custom
    '''
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.replace('.', '. ')
    text = re.sub('[^a-zA-Z.]', ' ', text)
    text = text.lower()
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.strip()

    stopwords_path = os.path.join(repo_root, "data/all_stop_words.txt")
    with open(stopwords_path, 'r') as f:
        stopwords = [line.strip() for line in f]
    
    dictionary = ArrayDictionary(stopwords)
    str = StopWordRemover(dictionary)
    text = str.remove(text) # 2x cleaning stop word
    text = str.remove(text)
    return text

def preprocess_tokenize(text):
    '''
    fungsi untuk memproses text yg sudah di process di preprocess() menjadi token
    dilakukan 3x preprocessing, karena stopword masih sering lewat kalau hanya 1x
    '''
    text = preprocess(text)
    text = preprocess(text)
    text = preprocess(text)
    tokens = text.split()
    tokens = [token for token in tokens if token]  # remove any empty tokens
    return tokens

In [ ]:
preprocessed_text = preprocess_tokenize(text_row)

4. Word2Vec Processing:

In [ ]:
def gen_word_embeddings(text_row, w2v_model):
    # Get the word embeddings for each word in the preprocessed text
    word_embeddings = {}
    for word in text_row:
        if word in w2v_model.wv:
            word_embeddings[word] = w2v_model.wv[word]
    return word_embeddings

In [ ]:
from gensim.models import Word2Vec

pred_w2v_tr_tune = pd.DataFrame()
w2v_path = os.path.join(repo_root, "models/w2v/idwiki_word2vec_100_new_lower.model")
w2v_model = Word2Vec.load(w2v_path)
for i in df.index:
    text_row = df['text'][i]
    unigram = extract_keywords(w2v_model, text_row, n=3).reset_index(drop=True)
    a = pd.DataFrame(unigram.keywords).T.reset_index(drop=True)
    b = pd.DataFrame(unigram.score).T.reset_index(drop=True)

    # add extra empty columns to a and b dataframes if necessary
    if a.shape[1] < 3:
        for i in range(3 - a.shape[1]):
            a[f'col{i+1}'] = ''
            b[f'col{i+1}'] = ''
    unigram = pd.concat([a, b], axis=1)
    if unigram.shape[1] < 6:
        for i in range(6 - unigram.shape[1]):
            unigram[f'col{i+1}'] = ''
    unigram.columns = ['key_1', 'key_2','key_3','score_1', 'score_2','score_3']

    pred_w2v_tr_tune = pd.concat([pred_w2v_tr_tune, unigram], ignore_index=True)
    pred_w2v_tr_tune[['score_1', 'score_2', 'score_3']] = pred_w2v_tr_tune[['score_1', 'score_2', 'score_3']].round(3)

pred_w2v_tr_tune

5. TextRank Integration:

6. Graph Construction with Word2Vec:

7. Extract Keyphrases:

8. Post-Processing & Results Display:

In [11]:
# import module dalam path utils?? belum berhasil
from utils import TextRank
from utils import LoadFile

#textrank = TextRank()
#loader = LoadFile()

pos = {'NN', 'VP', 'NP', 'FW'}
# 1. create a TextRank extractor.
extractor = TextRank()

In [6]:
import ast

df_keyphrases = pd.DataFrame(columns=['key_1', 'val_1', 'key_2', 'val_2', 'key_3', 'val_3'])

for text in df_pos:
    text1 = ast.literal_eval(text)
    extractor.load_document(input=text1) # text is not a list of list of tuple
    keyphrase = extractor.candidate_weighting_ia()

        # Separate the key and value elements into separate lists
    keys = [item[0] for item in keyphrase]
    values = [item[1] for item in keyphrase]

    # Create a dictionary with the keys and values
    data = {
    'key_1': [keys[0]],
    'val_1': [values[0]],
    'key_2': [keys[1]],
    'val_2': [values[1]],
    'key_3': [keys[2]],
    'val_3': [values[2]],
    }
    df_keyphrase = pd.DataFrame(data)
    
    # Concatenate df_keyphrase with df_keyphrases
    df_keyphrases = pd.concat([df_keyphrases, df_keyphrase], ignore_index=True)

In [7]:
# Write predictions to excel file
from utils import write_excel

sheet_name = 'w2v_pke_textrank'
output_file = 'w2v_pke_textrank.xlsx'
write_excel(df_keyphrases, sheet_name, output_file)

/home/jimx/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:15: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/home/jimx/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:24: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
